In [1]:
## Import packages

#Kspice
import sys
sys.path.append(r"C:\Program Files (x86)\Kongsberg\K-Spice\bin64") #add to path to allow kspice import
import kspice # if import error, check correct python version (3.11)

#Basic functionality
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#ML
import torch 

from enviroment import Sim
from DQN import DQN

In [2]:
#Initialize simulator

project_path = r"C:\Appl\K-Spice-Projects\Kristin_v23" #Specify path to downloaded project.
_ = kspice.Simulator(project_path) #Create instance of project
timeline = _.activate_timeline("Engineering") # Select the avaliable engineering timeline
app = "Topside" # We only make changes to the topside module NOTE: From software we can #deactivate Wells and Power in ESS model, can this be done from python? (If it increases speed)
timeline.initialize() #

In [3]:
#Load Kristin model, parameters and initial conditions

timeline.load_model("KristinMaria_master") #Load model
timeline.load_parameters("KristinMaria_master") # load fixed parameters
timeline.load_initial_condition("KristinMaria_master") # Load initial conditions

In [4]:
env = Sim(timeline, app) # Create env instance
env.import_variables("xl_tester1.xlsx") #Import variables of interest

In [7]:
env.check_df("Global KPI") #Check random column in dataframe. i.e. correct labeling in correspondance to simulator

In [8]:
env.timeline.set_speed(10) #Set speed of simulator, this value is what we wish for not necesarly what we get
env.timeline.run()

In [16]:
env.timeline.achieved_speed #Here you can see actual speed

1.0367207917313415

In [10]:
state = env.reset(pause = True) # We get reset the process simulator by pausing it and again fetching the initial conditions

In [13]:
def select_action(state):
    """ To ensure the model explores new spaces we will sometimes choose actions randomly. If not random we choose the action which result in the highest expected reward. 
    Choosing a random action will decay exponientially throughout learning. 
    """
    EPS_START = 0.9
    EPS_END = 0.05
    EPS_DECAY = 1000

    global steps_done
    sample = np.random.rand(1)
    eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor(env.sample("action"), dtype=torch.float32) # random action

In [11]:
#For the future we want the state at a specific form 
import itertools
state_flat = np.array(list(itertools.chain.from_iterable(state)))
state_tensor = torch.tensor(state_flat, dtype=torch.float32).unsqueeze(0) # make torch tensor and add axis

In [14]:
steps_done = 0
action = select_action(state_tensor) # select action (up, down, stay) for each controller by a factor of 1.
print(action)

tensor([-1., -1., -1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.])


In [15]:
observation, reward, terminated, truncated = env.step(action) # Add changes to model and use a polling function to check for updates, timeout currently as 5 min to reach new setpoint with arg: 0.1 precision

S23FIT1014 has reached its setpoint 12.993580287770818 [m3/h]
[0, 1, 2, 3, 4, 5, 7, 8, 9]
S23FIT1107 has reached its setpoint 30.02628758619506 [m3/h]
[0, 1, 2, 3, 4, 5, 7, 9]
S23TT1004 has reached its setpoint 29.094156591058947 [C]
[1, 2, 3, 4, 5, 7, 9]
S23LT1066 has reached its setpoint 42.09956626047431 [%]
[1, 2, 3, 4, 7, 9]
S23TT1034 has reached its setpoint 29.09475188628727 [C]
[1, 3, 4, 7, 9]
S23LT1006 has reached its setpoint 44.099918043728465 [%]
[3, 4, 7, 9]
Timeout reached


((array([2.89829813e+01, 4.39508368e+01, 2.89016007e+01, 4.52237861e+01,
         1.85913401e+01, 4.19254276e+01, 1.29328170e+01, 3.65216359e-05,
         2.98805423e+01, 1.75633582e-03]),
  array([ 7.36596831e+01, -1.88161242e+06, -8.18061287e+05, -1.12106806e+06]),
  array([29.        , 44.        , 29.        , 45.        , 21.        ,
         42.        , 13.        , 13.        , 30.00000191, 26.        ])),
 1,
 None,
 False)

In [17]:
env.timeline.pause() # Pause simulator